## Parallel Tool Use with Groq API

To extend the capabilities of Large Language Models (LLMs) in AI-powered applications and systems, we can provide tools to allow them to interact with external resources (e.g. APIs, databases, web) by:
    <ul>
    <li>Providing tools (or predefined functions) to our LLM</li>
    <li>Defining how the tools we provide should be used to teach our LLM how to use them effectively (e.g. defining input and output formats) </li>
    <li>
    Letting the LLM autonomously decide whether or not the provided tools are needed for a user query by evaluating the user query, determining whether the tools can enhance its response, and utilizing the tools accordingly</li>
    </ul>


##### Tool use, or function calling, support is available for all text models and parallel tool use support is enabled for all Llama 3 and Llama 3.1 models. The Llama 3.1 models now support the native tool use format that was used in post-training, which results in much better quality, especially in multi-turn conversations and parallel tool calling.

In [14]:
import os
import json

from groq import Groq
from dotenv import load_dotenv

load_dotenv()
"Groq API key configured: " + os.environ["GROQ_API_KEY"][:10] + "..."

'Groq API key configured: gsk_Et8zyU...'

In [15]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))
model = "llama-3.3-70b-versatile"

In [16]:
def get_bakery_prices(bakery_item: str):
    if bakery_item == "Cream Cake":
        return 4.25
    elif bakery_item == "dairymilk chocolate bar":
        return 2.50
    elif bakery_item == "Cup cake":
        return 4.75
    else:
        return "We're currently sold out!"

In [21]:
messages = [
    {"role": "system", "content": """You are a helpful assistant."""},
    {
        "role": "user",
        "content": "What is the price for a dairymilk chocolate bar and Cup cake?",
    },
]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_bakery_prices",
            "description": "Returns the prices for a given bakery product.",
            "parameters": {
                "type": "object",
                "properties": {
                    "bakery_item": {
                        "type": "string",
                        "description": "The name of the bakery item",
                    }
                },
                "required": ["bakery_item"],
            },
        },
    }
]
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
)

response_message = response.choices[0].message

In [22]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ray3', function=Function(arguments='{"bakery_item": "dairymilk chocolate bar"}', name='get_bakery_prices'), type='function'), ChatCompletionMessageToolCall(id='call_5r3r', function=Function(arguments='{"bakery_item": "Cup cake"}', name='get_bakery_prices'), type='function')])

In [23]:
tool_calls = response_message.tool_calls

messages.append(
    {
        "role": "assistant",
        "tool_calls": [
            {
                "id": tool_call.id,
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments,
                },
                "type": tool_call.type,
            }
            for tool_call in tool_calls
        ],
    }
)

available_functions = {
    "get_bakery_prices": get_bakery_prices,
}
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)

    # Note how we create a separate tool call message for each tool call
    # The model is able to discern the tool call result through `tool_call_id`
    messages.append(
        {
            "role": "tool",
            "content": json.dumps(function_response),
            "tool_call_id": tool_call.id,
        }
    )

print(json.dumps(messages, indent=2))

[
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "user",
    "content": "What is the price for a dairymilk chocolate bar and Cup cake?"
  },
  {
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_ray3",
        "function": {
          "name": "get_bakery_prices",
          "arguments": "{\"bakery_item\": \"dairymilk chocolate bar\"}"
        },
        "type": "function"
      },
      {
        "id": "call_5r3r",
        "function": {
          "name": "get_bakery_prices",
          "arguments": "{\"bakery_item\": \"Cup cake\"}"
        },
        "type": "function"
      }
    ]
  },
  {
    "role": "tool",
    "content": "2.5",
    "tool_call_id": "call_ray3"
  },
  {
    "role": "tool",
    "content": "4.75",
    "tool_call_id": "call_5r3r"
  }
]


In [24]:
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools, max_tokens=4096
)

print(response.choices[0].message.content)

The price for a dairymilk chocolate bar is 2.5 and the price for a Cup cake is 4.75.
